In [1]:
import tensorflow as tf
Adam = tf.keras.optimizers.Adam
to_categorical = tf.keras.utils.to_categorical
ImageDataGenerator = tf.keras.preprocessing.image.ImageDataGenerator
fashion_mnist = tf.keras.datasets.fashion_mnist
load_img = tf.keras.preprocessing.image.load_img
img_to_array = tf.keras.preprocessing.image.img_to_array

from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import plotly.graph_objs as go
from plotly import subplots
import plotly

import warnings
warnings.filterwarnings('ignore')

In [2]:
IMG_ROWS = 28
IMG_COLS = 28
NUM_CLASSES = 10
VAL_SIZE = 0.2
RANDOM_STATE = 99
BATCH_SIZE = 256

In [3]:
(train_data,y),(test_data,y_test) = fashion_mnist.load_data()

print("Fashion MNIST train - rows:",train_data.shape[0]," columns:",train_data.shape[1]," rows:",train_data.shape[2])
print("Fashion MNIST test - rows:",test_data.shape[0]," columns:",test_data.shape[1]," rows:",train_data.shape[2])

Fashion MNIST train - rows: 60000  columns: 28  rows: 28
Fashion MNIST test - rows: 10000  columns: 28  rows: 28


In [4]:
print(train_data.shape, test_data.shape)

train_data = train_data.reshape((train_data.shape[0], 28, 28, 1))
test_data = test_data.reshape((test_data.shape[0], 28, 28, 1))

print(train_data.shape, test_data.shape)

(60000, 28, 28) (10000, 28, 28)
(60000, 28, 28, 1) (10000, 28, 28, 1)


In [5]:
train_data = train_data/255.0
test_data = test_data/255.0

In [6]:
y = to_categorical(y)
y_test = to_categorical(y_test)

In [7]:
x_train, x_val, y_train, y_val = train_test_split(train_data, y, test_size=VAL_SIZE, random_state=RANDOM_STATE)

x_train.shape,x_val.shape,y_train.shape,y_val.shape

((48000, 28, 28, 1), (12000, 28, 28, 1), (48000, 10), (12000, 10))

In [8]:
datagen = ImageDataGenerator(
    rotation_range=10,  # เพิ่ม rotation
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1  # เพิ่ม shear
)
datagen.fit(x_train)

In [9]:
model = tf.keras.Sequential()

# 1. CNN LAYER
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding='Same', input_shape=(28,28,1)))
model.add(BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dropout(0.25))  # ลด Dropout

# 2. CNN LAYER
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding='Same'))
model.add(BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))

# 3. CNN LAYER
model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding='Same'))  # เพิ่ม filters
model.add(BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dropout(0.25))

# 4. CNN LAYER
model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding='Same'))  # เพิ่ม filters
model.add(BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))

# Fully Connected Layer
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256))
model.add(BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dropout(0.25))

# Output Layer
model.add(tf.keras.layers.Dense(10, activation='softmax'))

In [10]:
# ใช้ Adam พร้อม learning rate 0.0005
optimizer = Adam(learning_rate=0.0005)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 28, 28, 64)          │             640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 28, 28, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation (Activation)              │ (None, 28, 28, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 28, 28, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 28, 28, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 28, 28, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_1 (Activation)            │ (None, 28, 28, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 14, 14, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 14, 14, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_2 (Activation)            │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 14, 14, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 14, 14, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_3 (Activation)            │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 7, 7, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 7, 7, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 6272)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 1,870,026 (7.13 MB)

 Trainable params: 1,868,746 (7.13 MB)

 Non-trainable params: 1,280 (5.00 KB)

In [11]:
NO_EPOCH = 200
history = model.fit(datagen.flow(x_train, y_train, batch_size=BATCH_SIZE),
                    shuffle=True, epochs=NO_EPOCH, verbose=1,
                    validation_data=(x_val, y_val),
                    steps_per_epoch=x_train.shape[0] // BATCH_SIZE)

Epoch 1/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 38s 131ms/step - accuracy: 0.6235 - loss: 1.0433 - val_accuracy: 0.1521 - val_loss: 3.4411
Epoch 2/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7461 - loss: 0.6449 - val_accuracy: 0.1268 - val_loss: 3.4548
Epoch 3/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 16s 84ms/step - accuracy: 0.7753 - loss: 0.5864 - val_accuracy: 0.3049 - val_loss: 2.3042
Epoch 4/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8594 - loss: 0.3896 - val_accuracy: 0.2984 - val_loss: 2.3307
Epoch 5/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 19s 83ms/step - accuracy: 0.8098 - loss: 0.5010 - val_accuracy: 0.6522 - val_loss: 1.0434
Epoch 6/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8359 - loss: 0.4333 - val_accuracy: 0.6488 - val_loss: 1.0634
Epoch 7/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 16s 87ms/step - accuracy: 0.8228 - loss: 0.4692 - val_accuracy: 0.8637 - val_loss: 0.3604
Epoch 8/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8320 - loss: 0.4630 -

In [12]:
def create_trace(x,y,ylabel,color):
  # trace = go.Scatter(x=x, y=y, name=ylabel, marker=dict(color=color), mode='markers+lines', text=x)
  trace = go.Scatter(x=list(x), y=y, name=ylabel, marker=dict(color=color), mode='markers+lines', text=list(x))
  return trace

def plot_accuracy_and_loss(train_model):
  hist = train_model.history
  acc = hist['accuracy']
  val_acc = hist['val_accuracy']
  loss = hist['loss']
  val_loss = hist['val_loss']
  epochs = range(1,len(acc)+1)

  trace_ta = create_trace(epochs,acc,'Training Accuracy','green')
  trace_ts = create_trace(epochs,val_acc,'Validation Accuracy','red')
  trace_tl = create_trace(epochs,loss,'Training Loss','blue')
  trace_vl = create_trace(epochs,val_loss,'Validation Loss','magenta')

  fig = subplots.make_subplots(rows=1,cols=2,subplot_titles=('Training and Validation Accuracy','Training and Validation Loss'))
  fig.append_trace(trace_ta,1,1)
  fig.append_trace(trace_ts,1,1)
  fig.append_trace(trace_tl,1,2)
  fig.append_trace(trace_vl,1,2)
  fig['layout']['xaxis'].update(title='Epoch')
  fig['layout']['xaxis2'].update(title='Epoch')
  fig['layout']['yaxis'].update(title='Accuracy',range=[0,1])
  fig['layout']['yaxis2'].update(title='Loss',range=[0,1])

  plotly.offline.iplot(fig,filename='accuracy-loss')

In [13]:
plot_accuracy_and_loss(history)

In [14]:
score = model.evaluate(test_data, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.1943226009607315
Test accuracy: 0.9344000220298767
